In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import colors
import numpy as np
from collections import deque
import heapq

%matplotlib inline
plt.rcParams['animation.html'] = 'jshtml'  

class GridPathfinderMatplotlib:
    def __init__(self):
        # Grid parameters
        self.rows = 8
        self.cols = 10
        self.delay = 100  # milliseconds between frames
        
        # Initialize grid with walls from the image
        self.init_grid()
        
        # Setup the plot
        self.fig, self.ax = plt.subplots(figsize=(12, 10))
        self.setup_colormap()
        
        # Search state
        self.search_active = False
        self.algorithm_name = None
        self.steps_taken = 0
        self.goal_found = False
        self.algorithms = {
            'BFS': self.bfs,
            'DFS': self.dfs,
            'UCS': self.ucs,
            'DLS': self.dls,
            'IDDFS': self.iddfs,
            'Bidirectional': self.bidirectional
        }
        
        # For animation
        self.frames = []
        self.current_frame = 0
        
    def init_grid(self):
        """Initialize grid with walls from the image pattern"""
        # 0 represents free space, -1 represents walls
        self.grid = np.zeros((self.rows, self.cols))
        
        # Set start position (from image - 's' at bottom left)
        self.start = (6, 0)
        
        # Set target position (from image - 't' at row 6, col 7)
        self.target = (6, 7)
        
        # Set walls from image pattern (-1 in the grid)
        walls = [
            (1, 5), (2, 5), (3, 5), (4, 5),  # Vertical line of walls
            (4, 7)  # Single wall
        ]
        
        for r, c in walls:
            if 0 <= r < self.rows and 0 <= c < self.cols:
                self.grid[r, c] = -1  # -1 represents wall
                
        # Mark start and target with special values (these will be shown as S and T)
        self.grid[self.start] = 1  # Start
        self.grid[self.target] = 2  # Target
        
    def setup_colormap(self):
        """Setup custom colormap for visualization"""
        # Values: -1: wall, 0: empty, 1: start, 2: target, 
        # 3: frontier, 4: explored, 5: path
        self.cmap = colors.ListedColormap([
            'red',       # -1: wall
            'white',     # 0: empty (free space)
            'blue',      # 1: start
            'green',     # 2: target
            'yellow',    # 3: frontier
            'lightblue', # 4: explored
            'darkblue'   # 5: path
        ])
        
        self.bounds = [-1.5, -0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
        self.norm = colors.BoundaryNorm(self.bounds, self.cmap.N)
        
    def get_neighbors(self, pos):
        """Get valid neighbors in 6 directions: Up, Right, Bottom, Bottom-Right, Left, Top-Left"""
        r, c = pos
        # Movement order: Up, Right, Bottom, Bottom-Right (Diagonal), Left, Top-Left (Diagonal)
        directions = [
            (-1, 0),   # Up
            (0, 1),    # Right
            (1, 0),    # Bottom
            (1, 1),    # Bottom-Right (Diagonal)
            (0, -1),   # Left
            (-1, -1)   # Top-Left (Diagonal)
        ]
        
        neighbors = []
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if (0 <= nr < self.rows and 0 <= nc < self.cols and 
                self.grid[nr, nc] != -1):  # Not a wall
                neighbors.append((nr, nc))
        
        return neighbors
    
    def grid_to_graph(self):
        """Convert grid to graph format for bidirectional search"""
        graph = {}
        for i in range(self.rows):
            for j in range(self.cols):
                if self.grid[i, j] != -1:  # Not a wall
                    pos = (i, j)
                    graph[pos] = self.get_neighbors(pos)
        return graph
    
    def create_visualization_grid(self, frontier=None, explored=None, path=None):
        """Create a visualization grid with current state"""
        viz_grid = self.grid.copy()
        
        if frontier:
            for item in frontier:
                if isinstance(item, tuple):
                    if len(item) == 2:  # (node, path)
                        node = item[0]
                    elif len(item) == 3:  # (cost, node, path) for UCS
                        node = item[1]
                    else:
                        continue
                else:
                    node = item
                    
                if node != self.start and node != self.target:
                    viz_grid[node] = 3  # Frontier
        
        if explored:
            for node in explored:
                if node != self.start and node != self.target:
                    viz_grid[node] = 4  # Explored
        
        if path:
            for node in path:
                if node != self.start and node != self.target:
                    viz_grid[node] = 5  # Path
        
        return viz_grid
    
    def add_frame(self, viz_grid):
        """Add a frame to the animation"""
        self.frames.append(viz_grid.copy())
    
    def animate(self):
        """Animate the search process - returns animation object for Jupyter"""
        if not self.frames:
            print("No frames to animate")
            return None
        
        # Create a new figure for the animation
        fig, ax = plt.subplots(figsize=(12, 10))
        
        def update_frame(frame_num):
            ax.clear()
            viz_grid = self.frames[frame_num]
            
            # Display the grid
            ax.imshow(viz_grid, cmap=self.cmap, norm=self.norm)
            
            # Add text labels for S, T, and show 0 for free spaces, -1 for walls
            for i in range(self.rows):
                for j in range(self.cols):
                    val = int(viz_grid[i, j])
                    if val == 1:
                        ax.text(j, i, 'S', ha='center', va='center', color='white', fontweight='bold', fontsize=14)
                    elif val == 2:
                        ax.text(j, i, 'T', ha='center', va='center', color='white', fontweight='bold', fontsize=14)
                    elif val == -1:
                        ax.text(j, i, '-1', ha='center', va='center', color='black', fontsize=10, fontweight='bold')
                    elif val == 0:
                        ax.text(j, i, '0', ha='center', va='center', color='gray', fontsize=8, alpha=0.7)
                    elif val == 5:  # Path
                        ax.text(j, i, '●', ha='center', va='center', color='white', fontsize=12)
            
            # Formatting
            ax.set_xticks(np.arange(-0.5, self.cols, 1), minor=True)
            ax.set_yticks(np.arange(-0.5, self.rows, 1), minor=True)
            ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(f"{self.algorithm_name} - Step {frame_num + 1}")
        
        # Create the animation
        anim = animation.FuncAnimation(
            fig, update_frame, frames=len(self.frames), 
            interval=self.delay, repeat=False
        )
        
        plt.close(fig)  # Close the figure to prevent duplicate display
        return anim
    
    # BFS Implementation
    def bfs(self):
        """Breadth-First Search implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        # Initialize search state
        frontier = deque()
        frontier.append((self.start, [self.start]))
        frontier_set = {self.start}
        explored = set()
        
        current_grid = self.grid.copy()
        self.add_frame(current_grid)
        
        while frontier and self.search_active:
            self.steps_taken += 1
            current, path = frontier.popleft()
            frontier_set.remove(current)
            
            if current == self.target:
                # Show final path
                for node in path:
                    if node != self.start and node != self.target:
                        current_grid[node] = 5
                self.goal_found = True
                self.add_frame(current_grid)
                return
            
            if current not in explored:
                explored.add(current)
                if current != self.start and current != self.target:
                    current_grid[current] = 4  # Explored
                
                # Get neighbors
                for neighbor in self.get_neighbors(current):
                    if neighbor not in explored and neighbor not in frontier_set:
                        frontier.append((neighbor, path + [neighbor]))
                        frontier_set.add(neighbor)
                        if neighbor != self.start and neighbor != self.target:
                            current_grid[neighbor] = 3  # Frontier
                
                self.add_frame(current_grid)
        
        # No path found
        self.add_frame(current_grid)
    
    # DFS Implementation
    def dfs(self):
        """Depth-First Search implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        # Initialize search state
        frontier = [(self.start, [self.start])]  # Stack
        frontier_set = {self.start}
        explored = set()
        
        current_grid = self.grid.copy()
        self.add_frame(current_grid)
        
        while frontier and self.search_active:
            self.steps_taken += 1
            current, path = frontier.pop()
            frontier_set.remove(current)
            
            if current == self.target:
                for node in path:
                    if node != self.start and node != self.target:
                        current_grid[node] = 5
                self.goal_found = True
                self.add_frame(current_grid)
                return
            
            if current not in explored:
                explored.add(current)
                if current != self.start and current != self.target:
                    current_grid[current] = 4
                
                # Add neighbors in reverse for DFS
                neighbors = self.get_neighbors(current)
                for neighbor in reversed(neighbors):
                    if neighbor not in explored and neighbor not in frontier_set:
                        frontier.append((neighbor, path + [neighbor]))
                        frontier_set.add(neighbor)
                        if neighbor != self.start and neighbor != self.target:
                            current_grid[neighbor] = 3
                
                self.add_frame(current_grid)
        
        # No path found
        self.add_frame(current_grid)
    
    # UCS Implementation
    def ucs(self):
        """Uniform-Cost Search implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        # Initialize search state
        frontier = []  # Priority queue
        heapq.heappush(frontier, (0, self.start, [self.start]))
        frontier_dict = {self.start: 0}
        explored = set()
        
        current_grid = self.grid.copy()
        self.add_frame(current_grid)
        
        while frontier and self.search_active:
            self.steps_taken += 1
            cost, current, path = heapq.heappop(frontier)
            del frontier_dict[current]
            
            if current == self.target:
                for node in path:
                    if node != self.start and node != self.target:
                        current_grid[node] = 5
                self.goal_found = True
                self.add_frame(current_grid)
                return
            
            if current not in explored:
                explored.add(current)
                if current != self.start and current != self.target:
                    current_grid[current] = 4
                
                for neighbor in self.get_neighbors(current):
                    if neighbor not in explored:
                        # All moves have cost 1 (both cardinal and diagonal)
                        move_cost = 1
                        new_cost = cost + move_cost
                        
                        if neighbor not in frontier_dict or new_cost < frontier_dict[neighbor]:
                            heapq.heappush(frontier, (new_cost, neighbor, path + [neighbor]))
                            frontier_dict[neighbor] = new_cost
                            if neighbor != self.start and neighbor != self.target:
                                current_grid[neighbor] = 3
                
                self.add_frame(current_grid)
        
        # No path found
        self.add_frame(current_grid)
    
    # DLS Implementation
    def dls(self, limit=5):
        """Depth-Limited Search implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        current_grid = self.grid.copy()
        self.add_frame(current_grid)
        self.dls_recursive(self.start, [self.start], 0, limit, current_grid)
    
    def dls_recursive(self, node, path, depth, limit, current_grid):
        """Recursive DLS helper"""
        if not self.search_active:
            return True
        
        self.steps_taken += 1
        
        if node == self.target:
            for n in path:
                if n != self.start and n != self.target:
                    current_grid[n] = 5
            self.goal_found = True
            self.add_frame(current_grid)
            return True
        
        if depth >= limit:
            return False
        
        # Mark current as explored
        if node != self.start and node != self.target:
            current_grid[node] = 4
        
        for neighbor in self.get_neighbors(node):
            if neighbor not in path:
                if neighbor != self.start and neighbor != self.target:
                    current_grid[neighbor] = 3
                self.add_frame(current_grid)
                
                found = self.dls_recursive(neighbor, path + [neighbor], depth + 1, limit, current_grid)
                if found:
                    return True
        
        return False
    
    # IDDFS Implementation
    def iddfs(self, max_depth=20):
        """Iterative Deepening DFS implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        for depth in range(max_depth + 1):
            if not self.search_active:
                break
            
            current_grid = self.grid.copy()
            self.add_frame(current_grid)
            
            # Run DLS at current depth
            found = self.dls_recursive(self.start, [self.start], 0, depth, current_grid)
            if found:
                return
        
        if self.search_active:
            self.add_frame(self.grid.copy())
    
    # Bidirectional Search
    def bidirectional(self):
        """Bidirectional BFS implementation"""
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        current_grid = self.grid.copy()
        self.add_frame(current_grid)
        
        # Initial graph
        graph = self.grid_to_graph()
        
        path = self.bidirectional_bfs(graph, current_grid)
        
        if path:
            for node in path:
                if node != self.start and node != self.target:
                    current_grid[node] = 5
            self.goal_found = True
            self.add_frame(current_grid)
        else:
            self.add_frame(current_grid)
    
    def bidirectional_bfs(self, graph, current_grid):
        """Bidirectional BFS implementation"""
        if self.start == self.target:
            return [self.start]

        # Queues
        queue_f = deque([self.start])
        queue_b = deque([self.target])

        # Visited sets
        visited_f = {self.start}
        visited_b = {self.target}

        # Parents
        parent_f = {self.start: None}
        parent_b = {self.target: None}

        while queue_f and queue_b and self.search_active:
            self.steps_taken += 1
            
            # --- Forward BFS ---
            level_size = len(queue_f)
            for _ in range(level_size):
                if not queue_f:
                    break
                current = queue_f.popleft()
                
                for nbr in graph.get(current, []):
                    if nbr not in visited_f:
                        parent_f[nbr] = current
                        visited_f.add(nbr)
                        queue_f.append(nbr)
                        if nbr in visited_b:  # Meet check
                            return self.reconstruct_path(parent_f, parent_b, nbr)

            # --- Backward BFS ---
            level_size = len(queue_b)
            for _ in range(level_size):
                if not queue_b:
                    break
                current = queue_b.popleft()
                
                # For backward BFS, consider incoming edges
                for nbr, neighbors in graph.items():
                    if current in neighbors and nbr not in visited_b:
                        parent_b[nbr] = current
                        visited_b.add(nbr)
                        queue_b.append(nbr)
                        if nbr in visited_f:  # Meet check
                            return self.reconstruct_path(parent_f, parent_b, nbr)

        return None
    
    def reconstruct_path(self, parent_f, parent_b, meet):
        """Reconstruct path from bidirectional search"""
        # Forward path from start to meet
        path_f = []
        node = meet
        while node is not None:
            path_f.append(node)
            node = parent_f[node]
        path_f = path_f[::-1]  # reverse

        # Backward path from meet to goal
        path_b = []
        node = parent_b[meet]
        while node is not None:
            path_b.append(node)
            node = parent_b[node]

        return path_f + path_b
    
    def display_legend(self):
        """Display color legend"""
        legend_text = """
        COLOR LEGEND:
         Red       = Wall (-1)
         White     = Free Space (0)
         Blue      = Start (S)
         Green     = Target (T)
         Yellow    = Frontier (Nodes waiting to be explored)
         Light Blue = Explored (Nodes already visited)
         Dark Blue = Final Path
        """
        print(legend_text)
    
    def run_algorithm(self, algorithm_name):
        """Run the selected algorithm and display animation"""
        # Clear any previous output
        from IPython.display import clear_output
        clear_output(wait=True)
        
        self.search_active = True
        self.algorithm_name = algorithm_name
        self.frames = []
        self.steps_taken = 0
        self.goal_found = False
        
        # Display legend
        self.display_legend()
        
        print(f"\n▶ Running {algorithm_name}...")
        algorithm = self.algorithms[algorithm_name]
        algorithm()
        
        self.search_active = False
        
        # Print final result
        if self.goal_found:
            print(f"\n✅ GOAL FOUND! Steps taken: {self.steps_taken}")
        else:
            print(f"\n❌ GOAL NOT FOUND! Steps taken: {self.steps_taken}")
        
        if self.frames:
            print(f"\nGenerated {len(self.frames)} animation frames")
            anim = self.animate()
            
            # Display the animation directly
            from IPython.display import display
            display(anim)
            return anim
        else:
            print("No frames generated")
            return None


# Main execution for Jupyter
print("=" * 60)
print("AI Pathfinder - Uninformed Search Algorithms")
print("=" * 60)
print("\nGrid Configuration:")
print("   0 = Free space")
print("   -1 = Wall/Obstacle")
print("   S = Start (row 6, col 0)")
print("   T = Target (row 6, col 7)")
print("\nAvailable Algorithms:")
print("  1. BFS (Breadth-First Search)")
print("  2. DFS (Depth-First Search)")
print("  3. UCS (Uniform-Cost Search)")
print("  4. DLS (Depth-Limited Search)")
print("  5. IDDFS (Iterative Deepening DFS)")
print("  6. Bidirectional Search")
print("  0. Exit")

while True:
    try:
        choice = input("\nSelect algorithm (0-6): ").strip()
        
        if choice == '0':
            print("\nGoodbye!")
            break
        
        algorithms = {
            '1': 'BFS',
            '2': 'DFS',
            '3': 'UCS',
            '4': 'DLS',
            '5': 'IDDFS',
            '6': 'Bidirectional'
        }
        
        if choice in algorithms:
            algo_name = algorithms[choice]
            
            # Create new instance for each run
            pathfinder = GridPathfinderMatplotlib()
            pathfinder.run_algorithm(algo_name)
            
        else:
            print("  Invalid choice. Please try again.")
            
    except KeyboardInterrupt:
        print("\n\nGoodbye!")
        break
    except Exception as e:
        print(f"  Error: {e}")
        import traceback
        traceback.print_exc()


        COLOR LEGEND:
         Red       = Wall (-1)
         White     = Free Space (0)
         Blue      = Start (S)
         Green     = Target (T)
         Yellow    = Frontier (Nodes waiting to be explored)
         Light Blue = Explored (Nodes already visited)
         Dark Blue = Final Path
        

▶ Running BFS...

✅ GOAL FOUND! Steps taken: 41

Generated 42 animation frames
